In [1]:
import requests
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [2]:
baseurl = "https://api.wikiwho.net/en/api/v1.0.0-beta/"
content = "Mama's_Family"
def _url(url, path):
    return url + path
revisions_url = _url( baseurl,"rev_ids/")
mama_revisions_url = _url(revisions_url, content+"/")
params = {"editor": "true", "timestamp": "true"}
response = requests.get(mama_revisions_url, params= params)

In [3]:
response.url

"https://api.wikiwho.net/en/api/v1.0.0-beta/rev_ids/Mama's_Family/?editor=true&timestamp=true"

In [4]:
revisons = pd.DataFrame(response.json()["revisions"])
revisons_list = response.json()["revisions"]

In [5]:
revisons["id"].size

2547

In [6]:
rev_content_url = _url(baseurl,"rev_content/")
mama_rev_content_url = _url(rev_content_url, content+"/")
filename = content + ".h5"

In [7]:
all_content_ul = _url(baseurl, "all_content/" )
mama_all_content_url = _url(all_content_ul, content +"/")
params = { "o_rev_id": "true", "editor": "false", "token_id": "true", "in": "true", "out": "true" }
all_rev_data = requests.get(mama_all_content_url, params= params)


all_tokens_mama = all_rev_data.json()["all_tokens"]

In [ ]:
%%time
rev_contents = []
params ={ "o_rev_id": "false", "editor": "false", "token_id": "true","in": "false", "out": "false","str": "false" }
for i in range(15):
    url = _url(mama_rev_content_url, str(revisons.id.tolist()[i]) + "/")
    response = requests.get(url, params= params)
    rev_contents.append(response.json()["revisions"][0][str(revisons.id.tolist()[i])]["tokens"])

In [28]:
%%time
params ={ "o_rev_id": "false", "editor": "false", "token_id": "true","in": "false", "out": "false","str": "false" }

url = _url(mama_rev_content_url, str(revisons.id.tolist()[0]) + "/")
url = _url(url, str(revisons.id.tolist()[200]) + "/")

response = requests.get(url, params= params)

CPU times: user 352 ms, sys: 1.57 s, total: 1.92 s
Wall time: 4.68 s


In [18]:
response.json()["revisions"]

[]

#### Main cell to calculate the deleted and edited positions

In [ ]:
import time
params ={ "o_rev_id": "true", "editor": "false", "token_id": "true","in": "true", "out": "true" }
with pd.HDFStore(filename, 'a') as store:
    for revision in revisons_list:
        revision = str(revision["id"])
        url = _url(mama_rev_content_url, revision+"/")
        response = requests.get(url, params= params)
        if response.status_code == requests.codes.ok :
            df = pd.DataFrame.from_dict(response.json()["revisions"][0][revision]["tokens"])
#             df["inserted"] = df["in"].apply(lambda x: int(revision) in  set(x))
#             df["inserted"] = np.logical_or( df["inserted"].values, df["o_rev_id"].values == int(revision))
#             if(itr < (len(revisions) - 1) ):
#                 deleted_revision = revisions[itr + 1]
#             else :
#                 deleted_revision = -1
#             df["deleted"] = df["out"].apply(lambda x: deleted_revision in  set(x))
            store.put(revision, df, table=True)
        else:
            print(response.json())

In [ ]:
with pd.HDFStore( filename, 'r') as store1:
    df_3317807 = store1.get("3317807")
    df_3317866 = store1.get("3317866")
    print(df_3317807.head(2))
    print("\n")   
    print(df_3317866.head(2))

In [ ]:
inserted_array = np.array(df_3317866["inserted"])
inserted_pos = np.argwhere(inserted_array == True)
inserted_tokens = np.array(df_3317866 ["token_id"])[inserted_pos]

In [ ]:
# inserted_pos = inserted_pos[0].reshape(1,108)
inserted_pos.shape
# inserted_tokens.shape

In [ ]:
inserted_token_pos = np.stack((inserted_tokens, inserted_pos), axis = -1)
inserted_token_pos = inserted_token_pos.reshape(inserted_token_pos.shape[0],2)
inserted_token_pos

### To cluster elements using position of there relative position in doc

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=5, min_samples=1, p=1).fit(inserted_pos)
a = db.fit_predict(inserted_pos)
db.components_

In [ ]:
unique_clusters = set(db.labels_) - {-1}

In [ ]:
ls

In [ ]:
# rm Mama*

$$f'(a) = \lim_{x \to a} \frac{f(x) - f(a)}{x - a}$$

\begin{